In [ ]:
!pip install unidecode
import pandas as pd 
import numpy as np
import re
from unidecode import unidecode

def new_format(df, bts_col):
    new_format = []
    for i in df[bts_col]:
        if i == 'BTS':
            new_format.append('Yes')
        elif i == 'Legacy':
            new_format.append('No')
        else:
            new_format.append('')
    return new_format

def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index, date_cols):
    import pandas as pd
    import numpy as np

    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    df = pd.read_excel(path, sheet_name = sheetname, skiprows = n_skiprows)

    df = df.iloc[:,n_skip_columns:]
    df = df.dropna(subset=[site_index], axis=0)
    df.columns = lower_str(list(df.columns))
    
    #Create empty columns
    for i in range(1, 19):
        df[f'column{i}'] = [np.nan for i in range(df.shape[0])]
    
    # Parsing Dates
    for date_col in date_cols:
        lista = []
        df[date_col] = df[date_col].replace([' '], '')
        df[date_col] = df[date_col].fillna('')
        for i in df[date_col]:
            dates_format = re.match(r'\d{2}\/\d{2}\/\d{4}', str(i)) or re.match(r'\d{2}\-\d{2}\-\d{4}', str(i))
            not_date_format = not re.match(r'\d{2}\/\d{2}\/\d{4}', str(i)) or not re.match(r'\d{2}\-\d{2}\-\d{4}', str(i))
            if i !='' and not_date_format:
                #print(towerdb[towerdb[date_col]==i][['identification - site key', date_col]])
                lista.append(f'{i:%d/%m/%Y}')
            
            elif dates_format:
                lista.append(i)
            else:
                lista.append('')
        df[date_col] = lista

    return df

guideline = ["towerdb owner", "identification - site key", "identification - code", "identification - fl",\
             "position - site name", "identification - other mno fl", "position - macro region", \
             "position - region", "position - municipality", "position - address", "position - latitude", \
             "position - longitude", "category - categorization by transmission sys (subcluster)", \
             "category - categorization by site type", "column1", "category - categorisation by inhabitants", \
             "other - ownership", "other - status", "column2", "column3", "column4", "column5", "column6", \
             "column7", "column8", "column9", "column10", "lease contract - current annual lease fee", \
             "lease contract - lease contract comment", "column11", "column12", "column13", "column14", \
             "tenants - anchor", "tenants - vf - ran sharing", "tenants - vf - passive", \
             "tenants - wind - ran sharing", "tenants - wind - passive", "tenants - cosmote - passive", \
             "tenants - ran sharing tenants", "tenants - passive tenants", "tenants - non-mno tenants", \
             "tenants - non-mno tenant name", "tenants - total tenants", "column15", \
             "ee - no. of 24hr generators", "ee - no. of standby generators", "kpi - sites", "kpi - tenants", \
             "msa - bts/replacement", "msa - bts commitment site", "column16", \
             "infrastructure type - technology", "infrastructure type - fibre / microwave", \
             "infrastructure type - floor space", "infrastructure type - tower height", "column17",\
             "notified-97 (gr use only)", "changes (gr use only)", "msa - billing trigger stop date", \
             "other - indoor vs outdoor", "skylon - category", "msa - standard configuration", \
             "msa - construction type", "msa - hub sites total mw diameter", "bp - consolidated classification",\
             "msa - critical site", "sensitive - due to", "sensitive - description", "sensitive - department", \
             "notified - department", "notified - due to", "msa - sensitive", "msa - notified", \
             "notified - status", "notified - details", "msa - rectifier", "msa - battery", "msa - aircon", \
             "msa - revenue generating", "msa - date of decommissioning", "flag indicating bts site",\
             "msa - billing trigger date", "msa - dg-shelter", "msa - critical site in excess of the 15.5% cap",\
             "msa - form of active sharing", "msa - start date for active sharing arrangement",\
             "msa - end date for active sharing arrangement", "column18", "msa - rfai date", \
             "msa - site acceptance date", "msa - sub-lease reason", "msa - sub-lease applies", \
             "msa - date of equipment removal (vf)", "msa - date of equipment removal (wh)"]
guideline = list(map(lambda x: re.sub(r'^\s+|\s+$', '', str(x.lower())), guideline))

date_columns = ['msa - start date for active sharing arrangement', 'msa - rfai date', 'msa - site acceptance date', 'msa - billing trigger date', 
            'msa - date of decommissioning', 'msa - date of equipment removal (vf)', 'msa - date of equipment removal (wh)', 'msa - billing trigger stop date',
            'msa - end date for active sharing arrangement']

bill_cols = ['TowerDB owner','Identification - Site Key','Category - Categorisation by inhabitants','Other - Ownership','Other - Status',\
             'Lease Contract - Current annual lease fee','Tenants - VF - Passive','Tenants - Wind - Passive','Tenants - Cosmote - Passive',\
             'Tenants - Passive tenants','EE - No. of 24Hr generators','EE - No. of standby generators','Other - Indoor vs Outdoor',\
             'MSA - Standard Configuration','MSA - Construction Type','BP - Consolidated classification','MSA - Critical Site','MSA - Rectifier',\
             'MSA - Battery','MSA - Aircon','MSA - Billing Trigger Date','MSA - DG-Shelter','MSA - Critical Site in excess of the 15.5% cap',\
             'MSA - Form of Active Sharing','MSA - Start Date for Active Sharing Arrangement','MSA - Sub-Lease Applies','MSA - Date of Equipment Removal (VF)',\
             'MSA - Date of Equipment Removal (WH)']
bill_cols = list(map(lambda x: re.sub(r'^\s+|\s+$', '', str(x.lower())), bill_cols))

path_tw = "/content/GR_TowerDB Jun'21 FINAL (Billing version).xlsx"
sheet_tw = 'GR_TowerDB'

towerdb = read_files(path_tw, sheet_tw, 0, 0, 'Identification - Site Key', date_columns)
towerdb.rename(columns={'msa - legacy/bts': "flag indicating bts site"}, inplace=True)
towerdb.columns = list(map(lambda x: re.sub(r'^\s+|\s+$', '', str(x.lower())), towerdb.columns.to_list()))
towerdb = towerdb[guideline]
towerdb['flag indicating bts site'] = new_format(towerdb, 'flag indicating bts site')
towerdb['tenants - non-mno tenants'] = [int(i) if not pd.isnull(i) else '' for i in towerdb['tenants - non-mno tenants']]
towedb = towerdb.fillna('')

from unidecode import unidecode
df_old = pd.read_csv('/content/TowerDB_Greece_20210731.csv', encoding='windows-1252').fillna('')
df_old.columns = list(map(lambda x: re.sub(r'^\s+|\s+$', '', str(x.lower())), df_old.columns.to_list()))
#towerdb = towerdb.reindex(columns = lower_str(col_order))
df_old['identification - site key'] = df_old['identification - site key'].apply(unidecode)


In [ ]:
def comparison(df_OLD, df_NEW,index_col,bill_cols, path_save, old_name, new_name, status='', kind='tw',):
    # Perform Diff
    from unidecode import unidecode
    import pandas as pd
    from unidecode import unidecode
    import numpy as np
    import datetime as dt
    from datetime import datetime
    import re
    from openpyxl import Workbook, styles
    from openpyxl.styles import PatternFill, Font
    from openpyxl.styles.differential import DifferentialStyle
    from openpyxl.formatting.rule import Rule

    def fit_df(df, index_col):
        df = df.dropna(subset=[index_col], axis=0)
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    df_OLD = fit_df(df_OLD, index_col)
    df_NEW = fit_df(df_NEW, index_col)

    new_copy = df_NEW.copy()
    new_copy[index_col] = new_copy[index_col].apply(unidecode)
    droppedRows = []
    newRows = []

    cols_OLD = list(df_OLD.columns)
    cols_NEW = list(df_NEW.columns)
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))
    #print(sharedCols)
    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
            #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'Old value:{value_OLD} > New Value:{value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])

    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()

    if kind=='tw':
        sites = [i for i in new_copy['sites']] 
        old = df_OLD[[status]].reset_index()
        old = old.loc[old['sites'].isin(sites)]
        new = df_NEW[[status]].reset_index()
        new = new.loc[new['sites'].isin(sites)]
        df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
        new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
        status_1 = f'{status}_current'
        status_2 = f'{status}_before'
        new_copy = new_copy.set_index('sites')
        new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
        new_copy = new_copy.reset_index()

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format 
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

to_save_tw = '/content/GR_TW'
old_n = 'July.csv'
new_n = "August.xlsx"
index = 'Identification - Site Key'

comparison(df_old, towerdb, 'identification - site key',bill_cols, to_save_tw, old_n, new_n, 'other - status')


New Rows:     ['90642-Α64201', '90750-Α75002']
Dropped Rows: ['1135-113501', '1448-144801', '90642-A64201', '90750-A75002', '91250-D25002']

Done.



In [ ]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             path_save, old_name,new_name, type_file='mix', status_col='', kind='tw',kind_col='', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        kind = kind.lower()
        if kind == 'ta':
            df['sites'] = df[index_col].astype(str) + df[kind_col]

            #fit_cols = lower_str(list(df.columns))
            #df.columns = fit_cols
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            df = df.dropna(subset=[index_col], axis=0)
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = df_OLD.loc[row,col]
                    value_NEW = df_NEW.loc[row,col]
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'Old value:{value_OLD} > New Value:{value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old,  engine='python').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW,header=0, names=cols_new,  engine='python').fillna('')
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old, engine='python', encoding='windows-1252').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')
    
bill_cols = ['TowerDB owner',\
             'Identification - Site Key',\
             'Category - Categorisation by inhabitants',\
             'Other - Ownership',\
             'Other - Status',\
             'Lease Contract - Current annual lease fee',\
             'Tenants - VF - Passive',\
             'Tenants - Wind - Passive',\
             'Tenants - Cosmote - Passive',\
             'Tenants - Passive tenants',\
             'EE - No. of 24Hr generators',\
             'EE - No. of standby generators',\
             'Other - Indoor vs Outdoor',\
             'MSA - Standard Configuration',\
             'MSA - Construction Type',\
             'BP - Consolidated classification',\
             'MSA - Critical Site',\
             'MSA - Rectifier',\
             'MSA - Battery',\
             'MSA - Aircon',\
             'MSA - Billing Trigger Date',\
             'MSA - DG-Shelter',\
             'MSA - Critical Site in excess of the 15.5% cap',\
             'MSA - Form of Active Sharing',\
             'MSA - Start Date for Active Sharing Arrangement',\
             'MSA - End Date for Active Sharing Arrangement',\
             'MSA - Sub-Lease Applies',\
             'MSA - Date of Equipment Removal (VF)',\
             'MSA - Date of Equipment Removal (WH)']

in_month = '/content/TowerDB_Greece_20210731_renan.csv'
true_up = '/content/TowerDB_Greece_20210630.csv'
old_na = 'TowerDB_Greece_20210630.csv'
new_na = "TowerDB_Greece_20210731_renan.csv"
to_save = '/content/GR_TW'
index = 'Identification - Site Key'
find_diffs_between_files(true_up, in_month, index, bill_cols, to_save, old_na,new_na,'csv',status_col = 'Other - Status',kind='tw')

## UIS Comparisons

# Wind 

In [ ]:
!pip install unidecode
from unidecode import unidecode
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             path_save, old_name,new_name, type_file='mix', status_col='', kind='tw',kind_col='', sheetname='', skipr=0, skipc=0):
    import pandas as pd
    
    import numpy as np
    import datetime as dt
    from datetime import datetime
    import re
    from openpyxl import Workbook, styles
    from openpyxl.styles import PatternFill, Font
    from openpyxl.styles.differential import DifferentialStyle
    from openpyxl.formatting.rule import Rule
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        if kind == 'ta':
            df['sites'] = df[index_col].astype(str) + df[kind_col]

            #fit_cols = lower_str(list(df.columns))
            #df.columns = fit_cols
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            df = df.dropna(subset=[index_col], axis=0)
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = df_OLD.loc[row,col]
                    value_NEW = df_NEW.loc[row,col]
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old,  engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW,header=0, names=cols_new).fillna('')
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old, engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

uis_new = '/content/UserInput_Greece_WH_20210831.xlsx'
uis_old = '/content/UserInput_Greece_WH_20210731.xlsx'
sheet = 'SiteLevel'
uis_index = 'Site_ID (Alphanumeric or Numeric)'
to_uis = '/content/GR_UIS_WH'
old_uis = '20210731.xlsx'
new_uis = '20210831.xlsx'
bill = []
find_diffs_between_files(uis_old, uis_new, uis_index, bill, to_uis, old_uis,new_uis,'excel',status_col='',\
                         kind='',kind_col='', sheetname=sheet, skipr=2, skipc=0)

# Vodafone

In [ ]:
!pip install unidecode
from unidecode import unidecode
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             path_save, old_name,new_name, type_file='mix', status_col='', kind='tw',kind_col='', sheetname='', skipr=0, skipc=0):
    import pandas as pd
    
    import numpy as np
    import datetime as dt
    from datetime import datetime
    import re
    from openpyxl import Workbook, styles
    from openpyxl.styles import PatternFill, Font
    from openpyxl.styles.differential import DifferentialStyle
    from openpyxl.formatting.rule import Rule
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        if kind == 'ta':
            df['sites'] = df[index_col].astype(str) + df[kind_col]

            #fit_cols = lower_str(list(df.columns))
            #df.columns = fit_cols
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            df = df.dropna(subset=[index_col], axis=0)
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = df_OLD.loc[row,col]
                    value_NEW = df_NEW.loc[row,col]
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old,  engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW,header=0, names=cols_new).fillna('')
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old, engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

uis_new = '/content/UserInput_Greece_VF_20210831.xlsx'
uis_old = '/content/UserInput_Greece_VF_20210731.xlsx'
sheet = 'SiteLevel'
uis_index = 'Site_ID (Alphanumeric or Numeric)'
to_uis = '/content/GR_UIS_VF'
old_uis = '20210731.xlsx'
new_uis = '20210831.xlsx'
bill = []
find_diffs_between_files(uis_old, uis_new, uis_index, bill, to_uis, old_uis,new_uis,'excel',status_col='',\
                         kind='',kind_col='', sheetname=sheet, skipr=2, skipc=0)


New Rows:     ['1154-3010163', '124-1000124', '154-1000154', '156-1000156', '157-1000157', '177-1000177', '2134-1002134', '2193-1002193', '2316-1002316', '240-1000240', '244-1000244', '2648-2001218', '274-1000274', '279-1000279', '2913-2001217', '305-1000305', '306-2001304', '4986-3015543', '5531-3012398', '674-1000674', '867-1000867']
Dropped Rows: ['1027-3010382', '108-1000108', '1334-3007858', '146-1000146', '150-1000150', '2227-1002227', '224-2001202', '229-1000229', '2294-1002294', '2396-1002396', '391-2001302', '45-1000045', '4964-2001062', '5291-3011000', '5337-3015543', '5619-3010594', '6260-1006260', '695-1000695', '809-1000809', '817-2001118']

Done.



In [ ]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             path_save, old_name,new_name, type_file='mix', status_col='', kind='tw',kind_col='', sheetname='', skipr=0, skipc=0):
    import pandas as pd
    from unidecode import unidecode
    import numpy as np
    import datetime as dt
    from datetime import datetime
    import re
    from openpyxl import Workbook, styles
    from openpyxl.styles import PatternFill, Font
    from openpyxl.styles.differential import DifferentialStyle
    from openpyxl.formatting.rule import Rule
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        kind = kind.lower()
        if kind == 'ta':
            df['sites'] = df[index_col].astype(str) + df[kind_col]
            #fit_cols = lower_str(list(df.columns))
            #df.columns = fit_cols
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            df = df.dropna(subset=[index_col], axis=0)
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,index_col, status):
        # Perform Diff
        new_copy = df_NEW.copy()
        new_copy[index_col] = new_copy[index_col].apply(unidecode)
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = str(df_OLD.loc[row,col])
                    value_NEW = str(df_NEW.loc[row,col])
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'Old value:{value_OLD} > New Value:{value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old,  engine='python').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW,header=0, names=cols_new,  engine='python').fillna('')
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD, engine='python').fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        #towerdb = towerdb.reindex(columns = lower_str(col_order))
        df_OLD[index_col] = df_OLD[index_col].apply(unidecode)
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)
 
        """
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old, engine='python', encoding='windows-1252').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)"""

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)


    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW,index_col, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

bill_cols = ['TowerDB owner',\
             'Identification - Site Key',\
             'Category - Categorisation by inhabitants',\
             'Other - Ownership',\
             'Other - Status',\
             'Lease Contract - Current annual lease fee',\
             'Tenants - VF - Passive',\
             'Tenants - Wind - Passive',\
             'Tenants - Cosmote - Passive',\
             'Tenants - Passive tenants',\
             'EE - No. of 24Hr generators',\
             'EE - No. of standby generators',\
             'Other - Indoor vs Outdoor',\
             'MSA - Standard Configuration',\
             'MSA - Construction Type',\
             'BP - Consolidated classification',\
             'MSA - Critical Site',\
             'MSA - Rectifier',\
             'MSA - Battery',\
             'MSA - Aircon',\
             'MSA - Billing Trigger Date',\
             'MSA - DG-Shelter',\
             'MSA - Critical Site in excess of the 15.5% cap',\
             'MSA - Form of Active Sharing',\
             'MSA - Start Date for Active Sharing Arrangement',\
             'MSA - End Date for Active Sharing Arrangement',\
             'MSA - Sub-Lease Applies',\
             'MSA - Date of Equipment Removal (VF)',\
             'MSA - Date of Equipment Removal (WH)']
             
path_tw = "/content/GR_TowerDB Jun'21 FINAL (Billing version).xlsx"
msa = "/content/TowerDB_Greece_20210731.csv"
sheet_tw = 'GR_TowerDB'
skipr = 0
skipc = 0
to_save = '/content/GR_TW'
old_n = 'July.csv'
new_n = "August.xlsx"
index = 'Identification - Site Key'

find_diffs_between_files(msa, path_tw, index, bill_cols, to_save, old_n,new_n,'mix',\
                         status_col = 'Other - Status', kind='tw',kind_col='',sheetname=sheet_tw)


New Rows:     ['90642-Α64201', '90750-Α75002']
Dropped Rows: ['1135-113501', '1448-144801', '90642-A64201', '90750-A75002', '91250-D25002']

Done.

